In [44]:
import random
import numpy as np

We generate a sequence in the following data format:
* *x* and *y* describing a position in a grid of `100 x 100`
* *c* describing a control status with 3 possible states (1 = starting, 2 = holding, 3 = pausing)

State transitions follow this diagram:

```
+-+     +-+     +-+
|1| --> |2| --> |3|
+++  ^  +++  ^  +++
 ^   |   |   |   |
 |   |   |   |   |
 +---+---+   +---+
 |               |
 |               |
 +---------------+
```

This results in `100 x 100 x 3 = 30.000` possible one-hot encoded values.

In [61]:
STATE_STARTING = 1
STATE_HOLDING = 2
STATE_PAUSING = 3

DEFAULT_POSITION = [0, 0]


def random_position(grid_size):
    return [random.randint(1, grid_size) for _ in range(2)]


def next_state(previous_state):
    if previous_state == STATE_STARTING:
        next_state = STATE_HOLDING
    elif previous_state == STATE_HOLDING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_HOLDING,
            STATE_PAUSING
        ])
    elif previous_state == STATE_PAUSING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    else:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    return next_state
    

def generate_sequence(grid_size, length):
    current_state = None
    current_position = DEFAULT_POSITION
    for i in range(length):
        current_state = next_state(current_state)
        if current_state == STATE_STARTING:
            current_position = random_position(grid_size)
        elif current_state == STATE_PAUSING:
            current_position = DEFAULT_POSITION
        feature_vector = np.concatenate([current_position, [current_state]])
        print(feature_vector)

        
generate_sequence(100, 100)

[21 89  1]
[21 89  2]
[18 75  1]
[18 75  2]
[0 0 3]
[100  91   1]
[100  91   2]
[0 0 3]
[10 23  1]
[10 23  2]
[10 23  2]
[10 23  2]
[10 23  2]
[53 41  1]
[53 41  2]
[0 0 3]
[91 76  1]
[91 76  2]
[0 0 3]
[14  8  1]
[14  8  2]
[14  8  2]
[14  8  2]
[85 39  1]
[85 39  2]
[85 39  2]
[98 51  1]
[98 51  2]
[98 51  2]
[0 0 3]
[10 72  1]
[10 72  2]
[0 0 3]
[54 44  1]
[54 44  2]
[54 44  2]
[20 71  1]
[20 71  2]
[0 0 3]
[0 0 3]
[69  5  1]
[69  5  2]
[69  5  2]
[21 61  1]
[21 61  2]
[21 61  2]
[16 90  1]
[16 90  2]
[71  9  1]
[71  9  2]
[52 90  1]
[52 90  2]
[52 90  2]
[26 18  1]
[26 18  2]
[26 18  2]
[26 18  2]
[0 0 3]
[7 4 1]
[7 4 2]
[59 41  1]
[59 41  2]
[59 41  2]
[0 0 3]
[32  2  1]
[32  2  2]
[15 73  1]
[15 73  2]
[15 73  2]
[0 0 3]
[95 95  1]
[95 95  2]
[19 74  1]
[19 74  2]
[0 0 3]
[0 0 3]
[0 0 3]
[62 34  1]
[62 34  2]
[55 57  1]
[55 57  2]
[68 73  1]
[68 73  2]
[68 73  2]
[0 0 3]
[0 0 3]
[0 0 3]
[0 0 3]
[0 0 3]
[0 0 3]
[76 91  1]
[76 91  2]
[64 61  1]
[64 61  2]
[77 50  1]
[77 50  2]
[77 